In [1]:
# 1) Tell Gurobi where the license file is
ENV["GRB_LICENSE_FILE"] = raw"C:\Users\amanw\D\gurobi.lic"

# 2) Load packages AFTER the environment variable is set
using JuMP, Gurobi

# 3) Tiny sanity-check model
m = Model(Gurobi.Optimizer)
@variable(m, x >= 0)
@objective(m, Min, x)
optimize!(m)
println("Status = ", termination_status(m))


Set parameter Username
Set parameter LicenseID to value 2741171
Academic license - for non-commercial use only - expires 2026-11-19
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x84abb838
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00

User-callback calls 23, time in user-callback 0.00 sec
St

In [2]:
using Pkg
Pkg.add(["JuMP","Gurobi","PyCall","DataFrames","LinearAlgebra","Statistics","Plots","CSV"])


    Updating registry at `C:\Users\amanw\.julia\registries\General.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\amanw\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\amanw\.julia\environments\v1.12\Manifest.toml`


In [3]:
using Printf

const ROOT_DIR = @__DIR__
const OUT_DIR  = joinpath(ROOT_DIR, "gurobi_finance_data")

@printf("ROOT_DIR = %s\n", ROOT_DIR)
@printf("OUT_DIR  = %s\n", OUT_DIR)

if !isdir(OUT_DIR)
    error("Directory $OUT_DIR does not exist. Please unzip gurobi_finance_data.zip manually into this folder.")
else
    println("Data directory exists.")
end

function find_data_dir(root::AbstractString)
    for (r, _, files) in walkdir(root)
        if "mu.pkl" in files && "sigma.pkl" in files
            return r
        end
    end
    error("mu.pkl / sigma.pkl not found under $root.")
end

const DATA_DIR = find_data_dir(OUT_DIR)
println("DATA_DIR = ", DATA_DIR)


ROOT_DIR = d:\UDEM\IFT 6512
OUT_DIR  = d:\UDEM\IFT 6512\gurobi_finance_data
Data directory exists.
DATA_DIR = d:\UDEM\IFT 6512\gurobi_finance_data


In [4]:
using Pkg
Pkg.build("PyCall")   # optional, but good practice
using PyCall

pyimport_conda("pandas", "pandas")


    Building Conda ─→ `C:\Users\amanw\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\8f06b0cfa4c514c7b9546756dbae91fcfbc92dc9\build.log`
    Building PyCall → `C:\Users\amanw\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`


PyObject <module 'pandas' from 'C:\\Users\\amanw\\.julia\\conda\\3\\x86_64\\Lib\\site-packages\\pandas\\__init__.py'>

In [5]:
using PyCall, DataFrames, LinearAlgebra, Statistics

pd = pyimport("pandas")

mu_py    = pd.read_pickle(joinpath(DATA_DIR, "mu.pkl"))       # pandas Series
sigma_py = pd.read_pickle(joinpath(DATA_DIR, "sigma.pkl"))    # pandas DataFrame

assets = Vector{String}(mu_py.index.tolist())
μ       = Vector{Float64}(mu_py.values)
Σ       = Symmetric(Array{Float64}(sigma_py.values))

println("Number of risky assets: ", length(μ))
println("First 5 assets: ", assets[1:5])
println("μ (first 5) = ", μ[1:5])
println("Σ size = ", size(Σ))


Number of risky assets: 462
First 5 assets: ["APTV", "DVN", "HSY", "CAG", "HST"]
μ (first 5) = [0.23584795079384996, 0.20931364446446066, 0.20889248181352235, 0.16463680366755368, 0.16838619455355888]
Σ size = (462, 462)


In [ ]:
using Statistics

println("min μ = ", minimum(μ))
println("max μ = ", maximum(μ))
println("mean μ = ", mean(μ))


mean_annual_approx = mean(μ) * 52  
println("approx annual mean = ", mean_annual_approx, " % per year")


min μ = 0.07346582791256365
max μ = 0.5957955115760344
mean μ = 0.23327289150914832
approx annual mean = 12.130190358475712 % per year


1

In [8]:
# Add a risk-free asset with 2% annual return
rf_annual = 0.02
rf_weekly_pct = 100 * ((1 + rf_annual)^(1/52) - 1)   # convert to % per week

n_risky = length(μ)

assets = vcat(assets, "RF")
μ       = vcat(μ, rf_weekly_pct)

Σ = Symmetric([Matrix(Σ)          zeros(n_risky);
               zeros(1, n_risky)  0.0])

println("After adding RF:")
println("  Total assets     = ", length(μ))
println("  RF weekly return = ", rf_weekly_pct)
println("  Σ size           = ", size(Σ))


After adding RF:
  Total assets     = 463
  RF weekly return = 0.038089227674453774
  Σ size           = (463, 463)


2

In [ ]:
using JuMP
using Gurobi

# === Q2: Simple Markowitz variance–minimization model (min sigma^2) ===

n = length(μ)

# Required minimum expected return (in % per week, same unit as μ)
μ_bar = 0.5

model_minsigma = Model(Gurobi.Optimizer)

# Decision variables: x[i] = fraction of wealth invested in asset i
@variable(model_minsigma, x[1:n] >= 0)        # long-only; no short-selling

# Budget constraint: invest all wealth
@constraint(model_minsigma, sum(x) == 1)

# Minimum expected return constraint: μ' x ≥ μ_bar
@constraint(model_minsigma, dot(μ, x) >= μ_bar)

# Objective: minimize variance x' Σ x
@objective(model_minsigma, Min, dot(x, Σ * x))

optimize!(model_minsigma)

status_minsigma = termination_status(model_minsigma)
println("Model status = ", status_minsigma)

if status_minsigma == MOI.OPTIMAL
    x_star = value.(x)
    var_star = objective_value(model_minsigma)
    σ_star = sqrt(var_star)

    println("--------------------------------------------------")
    println("Required expected return (μ̄) = ", μ_bar, " % per week")
    println("Optimal variance           = ", var_star)
    println("Optimal standard deviation = ", σ_star)
    println("--------------------------------------------------")
    println("Non-zero positions:")
    for (name, w) in zip(assets, x_star)
        if w > 1e-4
            @printf("  %-8s : %.4f\n", name, w)
        end
    end
end


Set parameter Username
Set parameter LicenseID to value 2741171
Academic license - for non-commercial use only - expires 2026-11-19
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 2 rows, 463 columns and 926 nonzeros
Model fingerprint: 0x78e00249
Model has 106953 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-03, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve time: 0.01s
Presolved: 2 rows, 463 columns, 926 nonzeros
Presolved model has 106953 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 461
 AA' NZ     : 1.070e+05
 Factor NZ  : 1.074e+05 (roughly 1 MB of memory)
 Factor Ops : 3.319e+07 (less than 1 second p

3

In [ ]:
using JuMP
using Gurobi

# === Q3: Mean–Variance (utility) model: max μ'x − ½ γ x'Σx ===

n = length(μ)

# Risk aversion coefficient γ 
γ = 3.0              # for example; γ > 0

model_maxutil = Model(Gurobi.Optimizer)

# Decision variables: portfolio weights
@variable(model_maxutil, x[1:n] >= 0)        
@constraint(model_maxutil, sum(x) == 1)       

# Objective: maximize mean–variance utility
@objective(model_maxutil, Max, dot(μ, x) - 0.5 * γ * dot(x, Σ * x))

optimize!(model_maxutil)

status_maxutil = termination_status(model_maxutil)
println("Max-utility model status = ", status_maxutil)

if status_maxutil == MOI.OPTIMAL
    x_star_u = value.(x)

    # Compute expected return and variance of the optimal portfolio
    μ_star  = dot(μ, x_star_u)
    var_star = dot(x_star_u, Σ * x_star_u)
    σ_star  = sqrt(var_star)

    println("--------------------------------------------------")
    println("Risk aversion γ          = ", γ)
    println("Expected return μ(x*)    = ", μ_star,  " % per week")
    println("Variance      σ^2(x*)    = ", var_star)
    println("Std dev      σ(x*)       = ", σ_star)
    println("--------------------------------------------------")
    println("Non-zero positions in x*:")
    for (name, w) in zip(assets, x_star_u)
        if w > 1e-4
            @printf("  %-8s : %.4f\n", name, w)
        end
    end
end


Set parameter Username
Set parameter LicenseID to value 2741171
Academic license - for non-commercial use only - expires 2026-11-19
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1 rows, 463 columns and 463 nonzeros
Model fingerprint: 0xaa49f0e3
Model has 106953 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 6e-01]
  QObjective range [9e-03, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 1 rows, 463 columns, 463 nonzeros
Presolved model has 106953 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 461
 AA' NZ     : 1.065e+05
 Factor NZ  : 1.070e+05 (roughly 1 MB of memory)
 Factor Ops : 3.298e+07 (less than 1 second p

4

In [ ]:
using JuMP, Gurobi

# === Q4: Markowitz min-variance with diversification constraints ===

n = length(μ)

# Parameters for diversification (can be tuned)
μ_bar_div = 0.5      # required expected return 
K = 35               # minimal number of assets held
u = 0.15             # maximal position size for each asset
ℓ = 0.005            # minimal position size if an asset is held

model_div = Model(Gurobi.Optimizer)

# Positions: 0 <= x_i <= u
@variable(model_div, 0 <= x[1:n] <= u)

# Binary indicators: b_i = 1 if asset i is held, 0 otherwise
@variable(model_div, b[1:n], Bin)

# Budget: invest all wealth
@constraint(model_div, sum(x) == 1)

# Required expected return: μ' x ≥ μ̄
@constraint(model_div, dot(μ, x) >= μ_bar_div)

# Link x and b:
#  x_i = 0 if b_i = 0  (x_i ≤ b_i, since b_i ∈ {0,1} and u ≤ 1)
@constraint(model_div, [i=1:n], x[i] <= b[i])

#  x_i ≥ ℓ if asset i is held (b_i = 1)
@constraint(model_div, [i=1:n], x[i] >= ℓ * b[i])

# Diversification: at least K assets must be held
@constraint(model_div, sum(b) >= K)

# Objective: minimize portfolio variance x' Σ x
@objective(model_div, Min, dot(x, Σ * x))

optimize!(model_div)

status_div = termination_status(model_div)
println("Diversification model status = ", status_div)

if status_div == MOI.OPTIMAL
    x_star_div = value.(x)
    b_star_div = value.(b)

    var_star_div = objective_value(model_div)
    σ_star_div   = sqrt(var_star_div)
    μ_star_div   = dot(μ, x_star_div)

    println("--------------------------------------------------")
    println("Required return μ̄      = ", μ_bar_div, " % per week")
    println("Realized μ(x*)          = ", μ_star_div, " % per week")
    println("Variance σ^2(x*)        = ", var_star_div)
    println("Std dev σ(x*)           = ", σ_star_div)
    println("Number of active assets = ", sum(b_star_div .> 0.5))
    println("--------------------------------------------------")
    println("Non-zero positions (x_i):")
    for (name, w, bb) in zip(assets, x_star_div, b_star_div)
        if bb > 0.5     # held
            @printf("  %-8s : %.4f\n", name, w)
        end
    end
end


Set parameter Username
Set parameter LicenseID to value 2741171
Academic license - for non-commercial use only - expires 2026-11-19
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 929 rows, 926 columns and 3241 nonzeros
Model fingerprint: 0x6835335d
Model has 106953 quadratic objective terms
Variable types: 463 continuous, 463 integer (463 binary)
Coefficient statistics:
  Matrix range     [5e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-03, 2e+02]
  Bounds range     [1e-01, 1e-01]
  RHS range        [5e-01, 4e+01]
Presolve time: 0.03s
Presolved: 929 rows, 926 columns, 3240 nonzeros
Presolved model has 106953 quadratic objective terms
Variable types: 463 continuous, 463 integer (463 binary)

Root relaxation: objective 1.782815e+01, 1501 iterations,

5

In [ ]:
using DataFrames

# Helper: compute summary statistics for a given portfolio x
function portfolio_stats(name::AbstractString, x::Vector{Float64},
                         μ::Vector{Float64}, Σ::Symmetric{Float64, Matrix{Float64}};
                         thresh::Float64 = 1e-5)
    @assert length(x) == length(μ) == size(Σ,1)

    μ_p   = dot(μ, x)              # expected return (% per week)
    var_p = dot(x, Σ * x)          # variance
    σ_p   = sqrt(var_p)            # standard deviation
    n_act = count(>(thresh), x)    # number of active positions
    rf_w  = x[end]                 # weight of risk-free asset 

    return (name = name,
            exp_return = μ_p,
            variance   = var_p,
            stdev      = σ_p,
            n_active   = n_act,
            rf_weight  = rf_w)
end

# === 1) Portfolios already computed in previous questions ===

rows = NamedTuple[]

# Q2: min-variance with return floor (x_star)
push!(rows, portfolio_stats("MinVar (Q2)", x_star, μ, Σ))

# Q3: mean–variance utility max (x_star_u)
push!(rows, portfolio_stats("MaxUtility (Q3)", x_star_u, μ, Σ))

# Q4: min-variance + diversification constraints (x_star_div)
push!(rows, portfolio_stats("MinVar+Diversification (Q4)", x_star_div, μ, Σ))

# 2) Uniform portfolio 
n = length(μ)
x_unif = fill(1.0 / n, n)
push!(rows, portfolio_stats("Uniform", x_unif, μ, Σ))

# 3) Max-return portfolio ignoring risk 
model_maxret = Model(Gurobi.Optimizer)
@variable(model_maxret, x_ret[1:n] >= 0)
@constraint(model_maxret, sum(x_ret) == 1)
@objective(model_maxret, Max, dot(μ, x_ret))
optimize!(model_maxret)

x_star_ret = value.(x_ret)
push!(rows, portfolio_stats("MaxReturn (no risk)", x_star_ret, μ, Σ))

df_compare = DataFrame(rows)

println(df_compare)


Set parameter Username
Set parameter LicenseID to value 2741171
Academic license - for non-commercial use only - expires 2026-11-19
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1 rows, 463 columns and 463 nonzeros
Model fingerprint: 0x0bc5060b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 463 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.9579551e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.957955116e-01

User-callback calls 37, time in user-callback 0.00 

6

In [ ]:
using Pkg
Pkg.add("Distributions")   

using Distributions


   Resolving package versions...
   Installed PDMats ────────────────── v0.11.36
   Installed Rmath_jll ─────────────── v0.5.1+0
   Installed Rmath ─────────────────── v0.9.0
   Installed StatsFuns ─────────────── v1.5.2
   Installed HypergeometricFunctions ─ v0.3.28
   Installed FillArrays ────────────── v1.15.0
   Installed QuadGK ────────────────── v2.11.2
   Installed Distributions ─────────── v0.25.122
  Installing 1 artifacts
   Installed artifact Rmath                 202.0 KiB
    Updating `C:\Users\amanw\.julia\environments\v1.12\Project.toml`
  [31c24e10] + Distributions v0.25.122
    Updating `C:\Users\amanw\.julia\environments\v1.12\Manifest.toml`
  [31c24e10] + Distributions v0.25.122
  [1a297f60] + FillArrays v1.15.0
  [34004b35] + HypergeometricFunctions v0.3.28
  [90014a1f] + PDMats v0.11.36
  [1fd47b50] + QuadGK v2.11.2
  [79098fc4] + Rmath v0.9.0
  [4c63d2b9] + StatsFuns v1.5.2
  [f50d1b31] + Rmath_jll v0.5.1+0
  [4607b0f0] + SuiteSparse
Precompiling packages...
    6

In [14]:
# Helper: for a Normal(μ, σ²) return, compute P(R<0) and ES = -E[R|R<0]
function prob_neg_and_es(μ_p::Float64, σ_p::Float64)
    if σ_p < 1e-10
        # Almost deterministic portfolio
        if μ_p < 0
            return 1.0, -μ_p     # always negative; loss is just -μ
        else
            return 0.0, 0.0      # never negative; ES = 0
        end
    end

    z  = (0.0 - μ_p) / σ_p
    Φz = cdf(Normal(0, 1), z)
    ϕz = pdf(Normal(0, 1), z)

    p_neg = Φz                   # P(R < 0)

    if Φz < 1e-12
        # Prob of negative return essentially zero → ES ≈ 0
        return p_neg, 0.0
    end

    cond_mean = μ_p - σ_p * ϕz / Φz   # E[R | R < 0] (negative)
    es_loss   = -cond_mean            # positive expected loss

    return p_neg, es_loss
end

# Apply to each portfolio in df_compare
p_list  = Float64[]
es_list = Float64[]

for row in eachrow(df_compare)
    μ_p = row.exp_return
    σ_p = row.stdev
    p_neg, es_loss = prob_neg_and_es(μ_p, σ_p)
    push!(p_list,  p_neg)
    push!(es_list, es_loss)
end

df_compare[!, :prob_neg]       = p_list
df_compare[!, :ES_given_rneg]  = es_list

println(df_compare)


5×8 DataFrame
 Row │ name                         exp_return  variance     stdev       n_active  rf_weight    prob_neg  ES_given_rneg 
     │ String                       Float64     Float64      Float64     Int64     Float64      Float64   Float64       
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ MinVar (Q2)                    0.5        14.679       3.83132           8  1.27823e-14  0.448084       2.88221
   2 │ MaxUtility (Q3)                0.046818    0.0029096   0.0539407        32  0.968134     0.19271        0.029801
   3 │ MinVar+Diversification (Q4)    0.5        17.8281      4.22234          35  0.0          0.452868       3.19357
   4 │ Uniform                        0.232851    4.80307     2.19159         463  0.00215983   0.457693       1.66667
   5 │ MaxReturn (no risk)            0.595796  117.997      10.8627            1  0.0          0.47813        8.45418


7

- **Portefeuille MaxReturn (sans risque)**  
  C’est le portefeuille « myope » qui maximise μ(x) sans pénaliser le risque.  
  Il délivre le **rendement espéré le plus élevé (≈ 0.60%/semaine)** mais au prix d’une **volatilité très importante (σ ≈ 10.9)**, d’une **probabilité de rendement négatif proche de 50%** et d’une **perte espérée conditionnelle élevée (ES ≈ 8.45)**.  
  On obtient ainsi un portefeuille extrêmement concentré (1 seul actif), très risqué et peu diversifié.

- **Portefeuille MinVar (Q2)**  
  À l’autre extrême, on impose un niveau de rendement cible μ̄ = 0.5%/semaine et on minimise la variance.  
  On obtient un rendement exactement à la cible, mais avec **σ ≈ 3.83**, une probabilité de rendement négatif d’environ **45%** et une **ES ≈ 2.88**.  
  Le portefeuille n’utilise que quelques actifs (8) et ne fait pratiquement pas appel à l’actif sans risque, ce qui traduit une solution « frontière efficiente » avec peu de titres mais un profil de risque modéré.

- **Portefeuille MaxUtility (Q3)**  
  Le modèle moyenne–variance $\max \mu(x) - \tfrac{1}{2}\gamma\, \sigma^2(x)$ avec $\gamma = 3$ aboutit à un portefeuille **très conservateur** : le poids sur l’actif sans risque est ≈ 97%.  
  Le rendement espéré est faible (≈ 0.05%/semaine), mais **la volatilité est quasi nulle (σ ≈ 0.054)**, la **probabilité de perte tombe à ≈ 19%** et **l’ES est négligeable (≈ 0.03)**.  
  Ce portefeuille illustre bien qu’un investisseur très avers au risque privilégiera massivement l’actif sans risque et acceptera un rendement très bas.

- **Portefeuille MinVar + Diversification (Q4)**  
  On impose ici des contraintes de diversification (taille min/max des positions et nombre minimal d’actifs).  
  Par construction, le rendement espéré reste à **0.5%/semaine**, mais la variance augmente par rapport au MinVar de base (**σ ≈ 4.22 vs 3.83**), et l’ES passe de ≈ 2.88 à ≈ 3.19.  
  En contrepartie, **le nombre d’actifs détenus passe de 8 à 35**, ce qui réduit le risque de concentration et rend le portefeuille plus robuste à des chocs idiosyncratiques.  
  Les contraintes de diversification déplacent donc le portefeuille le long de la frontière efficiente vers un point un peu moins efficient, mais structurellement plus diversifié.

- **Portefeuille uniforme**  
  Le portefeuille 1/N fournit un repère « naïf » : il est **très diversifié (463 actifs)**, mais n’est ni optimal en rendement ni en risque.  
  Il affiche un rendement moyen intermédiaire (≈ 0.23%/semaine) et une volatilité (σ ≈ 2.19) plus faible que MinVar (Q2) car il se situe implicitement plus près d’un point de la frontière efficiente avec objectif de variance uniquement.  
  Cependant, la probabilité de rendement négatif (≈ 46%) et l’ES (≈ 1.67) restent significatives, ce qui montre qu’une simple diversification naïve ne remplace pas une optimisation explicite.

### Conclusion

- Le **portefeuille MaxReturn** illustre le cas « agressif » : rendement maximal mais risque (σ, probabilité de perte et ES) très élevé et concentration extrême.  
- Le **portefeuille MaxUtility** est à l’opposé : rendement très faible mais risque quasi nul grâce à un recours massif à l’actif sans risque.  
- Les portefeuilles **MinVar** et **MinVar + diversification** offrent des compromis plus équilibrés entre rendement et risque, le second sacrifiant un peu d’efficience pour une meilleure diversification.  
- Le **portefeuille uniforme** se situe entre ces solutions mais reste dominé en termes de couple (μ, σ) et de gestion du risque de pertes extrêmes, ce qui justifie l’intérêt des modèles d’optimisation plutôt que d’une simple pondération 1/N.

### Choix des paramètres des modèles

Dans les différentes questions, plusieurs paramètres numériques doivent être fixés.  
Ils n’ont pas été imposés dans l’énoncé; nous les choisissons comme suit.

#### Actif sans risque (Questions 1–7)

- Taux annuel fixé à 2% (donné dans l’énoncé).  
- Pour être cohérent avec les données de Gurobi (rendements en pourcentage par semaine),  
  nous convertissons ce taux annuel en rendement hebdomadaire :

  rf_hebdo = 100 * [ (1 + 0.02)^(1/52) - 1 ]  ≈ 0.038 % par semaine

- Cet actif est ajouté comme un actif supplémentaire avec variance nulle et covariances nulles.

#### Q2 – Modèle de Markowitz : minimisation de variance

- Niveau de rendement cible fixé à 0.5 % par semaine.
- Cette valeur est choisie pour être :
  - significativement supérieure au rendement de l’actif sans risque,
  - mais inférieure aux rendements extrêmes de certaines actions individuelles.
  Cela permet d’obtenir un portefeuille sur la frontière efficiente mais encore diversifié.

#### Q3 – Modèle moyenne–variance (maximisation d’utilité)

- Le critère d’utilité utilisé est :

  maximiser :   mu · x  –  (1/2) * gamma * (x' Σ x)

- Nous fixons gamma = 3.
- Ce choix donne un portefeuille très prudent (poids élevé sur l’actif sans risque),
  ce qui illustre bien le compromis rendement–risque quand gamma augmente.

#### Q4 – Contraintes de diversification

Nous introduisons des variables binaires bi et les contraintes suivantes :

- **Nombre minimal d’actifs :**

  somme(bi) ≥ K  avec K = 35

- **Bornes individuelles sur les poids :**

  l * bi  ≤  xi  ≤  u * bi

  avec l = 0.5 % et u = 15 %.

  - u = 15% empêche qu’un seul titre dépasse 15% du portefeuille.  
  - l = 0.5% évite des positions négligeables lorsque bi = 1.

#### Portefeuilles de référence (Question 5)

- **Portefeuille uniforme :**

  xi = 1 / n  pour tout i

- **Portefeuille “MaxReturn (sans risque)” :**

  maximiser : mu · x  
  sous contraintes : somme(xi) = 1 et xi ≥ 0

  Ce portefeuille ignore totalement la variance et choisit l’actif au rendement espéré maximum.


